<a href="https://colab.research.google.com/github/Nmane1612/Nihar-Mane/blob/master/Stock%20Market%20Analysis%20%26%20Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Stock Market Analysis & Prediction (Data Warehousing & Mining project)***

In [ ]:
#Libraries To Be Imported
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
import tensorflow
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense , LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [ ]:
#Geting Stock Data 
df = web.DataReader('AAPL', data_source = 'yahoo', start = '2012-01-01', end = '2019-12-12')
#Showing Stock Data
df

In [ ]:
#Getting Number Of Rows & Columns From The Data
df.shape

In [ ]:
#Visualise Closing Price
plt.figure(figsize=(16,8))
plt.title('Closing Price')
plt.plot(df['Close'])
plt.xlabel('Date', fontsize = 20)
plt.ylabel('Close Price INR (₹)', fontsize = 20)
plt.show()

In [ ]:
#Creating Close Column DataFrame
data = df.filter(['Close'])
#Convert The DataFrame To A Numpy Array
dataset = data.values
#Get Number Of Rows To Train The Model On
training_data_len = math.ceil(len(dataset)*.9)
training_data_len

In [ ]:
#Scaling Data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
scaled_data

In [ ]:
#Creating Trained Data Set
#Creating Scaled Training Data Set
train_data = scaled_data[0:training_data_len , :]
#Splitting Data Into x_train & y_train Data Sets
x_train = []
y_train = []
for i in range(60,len(train_data)):
  x_train.append(train_data[i-60:i,0])
  y_train.append(train_data[i,0])
  if i<=60:
    print(x_train)
    print(y_train)
    print()

In [ ]:
#Convert x_train & y_train Dataset To Numpy Arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshaping Data
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_train.shape

In [ ]:
#Building Long Short-Term Memory(LSTM) Model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
#Compiling The Model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
#Training The Model
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
#Create Testing Dataset
#Create New Array Containing Scaled Values From index 1740 To 2000
test_data = scaled_data[training_data_len - 60: , :]
#Now Creating Data Sets x_test & y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
  x_test.append(test_data[i-60:i, 0])

In [ ]:
#Converting The Data Into A Numpy Array
x_test = np.array(x_test)

In [ ]:
#Reshaping The Data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
#Get Models Predicted Price Values
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#Get The Root Mean Squared Error (RMSE)
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
#Plot The Data
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions
#Visualizing The Data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=20)
plt.ylabel('Close Price INR (₹)', fontsize = 20)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'], loc='upper left')
plt.show()

In [ ]:
#Show The Valid & Predicted Prices
valid

In [ ]:
#Get Quote
apple_quote = web.DataReader('AAPL', data_source = 'yahoo', start = '2012-01-01', end = '2019-12-12')
#Create New Dataframe
new_df = apple_quote.filter(['Close'])
#Get The Last 60 Day Closing Price Values & Convert The DataFrame To An Array
last_60_days = new_df[-60:].values
#Scale The Data To Be Values Between 0 & 1
last_60_days_scaled = scaler.transform(last_60_days)
#Creating An Empty List
X_test = []
#Append Past 60 Days
X_test.append(last_60_days_scaled)
#Convert X_test Data Set To A Numpy Array
X_test = np.array(X_test)
#Reshape The Data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get Predicted Scaled Price
pred_price = model.predict(X_test)
#Undo Scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
#Get Quote
apple_quote1 = web.DataReader('AAPL', data_source='yahoo', start='2019-12-12', end='2019-12-12')
print(apple_quote1['Close'])